In [ ]:
import pandas as pd
import numpy as np
from math import pi
import warnings
import matplotlib.pyplot as plt
import plotly.express as px

# Downloading dataset containing Emissions, GDP and Population

Warning: The data for Arsenic (As), Nickel (Ni) and Chromium (Cr) emissions for Switzerland, Luxembourg and Austria were missing so I set these values to 0, because these emissions don't really affect the results

In [ ]:
Data = pd.read_csv('https://raw.githubusercontent.com/AntoineTrabia/Green-Domestic-Product/main/data/Data_for_GrDP_territorial.csv')

# Marginal Damage Cost of pollutants

## Information on dataset
GHG: Euro per tCO2eq

Health 1: Impact on health from fine particulate matter and ozone
Euro per tonne of emissions of precursors

Health 2: Impact on health from NO2
Euro per tonne of emissions of precursors

Crops: Impact on crops from ozone
Euro per tonne of emissions of precursors

Forests: Impact on forests from ozone
Euro per tonne of emissions of precursors

Buildings: Impact on buildings
Euro per tonne of emissions

Ecosystems: Impact on ecosystems (eutrophication)
Euro per tonne of emissions  of precursors

Heavy metals: Euro per kg of pollutant

In [ ]:
MDC = pd.read_excel('https://raw.githubusercontent.com/AntoineTrabia/Green-Domestic-Product/main/data/MDC.xlsx')

In [ ]:
# We convert the unit into euro per tonne of pollutant for the heavy metals
heavy_metals = ['Pb cost', 'Cd cost', 'Hg cost', 'As cost', 'Ni cost','Cr cost']
for metal in heavy_metals:
  column = metal + ' [Euro/kg]'
  new_column = metal + ' [Euro/t]'
  MDC[column] = 1000*MDC[column]
  MDC.rename(columns={column: new_column},inplace=True)

# External Costs

In [ ]:
Data_for_calc = pd.merge(Data, MDC, on=['countries'])
External_costs = Data_for_calc.loc[:,['countries','Year']]

## External cost due to impacts of climate change (GHG emissions)

### Low Marginal Cost Scenario

In [ ]:
External_costs['Low GHG [Euro]'] = Data_for_calc['GHG [t]'] * Data_for_calc['Low GHG [Euro/t]']

### Central Margical Cost Scenario

In [ ]:
External_costs['Central GHG [Euro]'] = Data_for_calc['GHG [t]'] * Data_for_calc['Central GHG [Euro/t]']

### High Marginal Cost Scenario

In [ ]:
External_costs['High GHG [Euro]'] = Data_for_calc['GHG [t]'] * Data_for_calc['High GHG [Euro/t]']

## External costs due to nitrogen oxides (NOx) emissions

In [ ]:
External_costs['Nox (Health - VOLY) [Euro]'] = Data_for_calc['NOx [t]'] * (Data_for_calc['Health 1: Nox VOLY [Euro/t]'] + Data_for_calc['Health 2: NOx VOLY [Euro/t]'])
External_costs['Nox (Health - VSL) [Euro]'] = Data_for_calc['NOx [t]'] * (Data_for_calc['Health 1: Nox VSL [Euro/t]'] + Data_for_calc['Health 2: NOx VSL [Euro/t]'])
External_costs['Nox (Crops, Forests, Buildings & Ecosystems) [Euro]'] = Data_for_calc['NOx [t]'] * (Data_for_calc['Crops: Nox [Euro/t]'] + Data_for_calc['Buildings: Nox [Euro/t]'] + Data_for_calc['Ecosystems: Nox [Euro/t]'] + Data_for_calc['Forests: Nox [Euro/t]'])
External_costs['Nox (Crops) [Euro]'] = Data_for_calc['NOx [t]'] * Data_for_calc['Crops: Nox [Euro/t]']
External_costs['Nox (Forests) [Euro]'] = Data_for_calc['NOx [t]'] * Data_for_calc['Forests: Nox [Euro/t]']
External_costs['Nox (Buildings) [Euro]'] = Data_for_calc['NOx [t]'] * Data_for_calc['Buildings: Nox [Euro/t]']
External_costs['Nox (Ecosystems) [Euro]'] = Data_for_calc['NOx [t]'] * Data_for_calc['Ecosystems: Nox [Euro/t]']
External_costs['Nox (Total - VOLY) [Euro]'] = External_costs['Nox (Health - VOLY) [Euro]'] + External_costs['Nox (Crops, Forests, Buildings & Ecosystems) [Euro]']
External_costs['Nox (Total - VSL) [Euro]'] = External_costs['Nox (Health - VSL) [Euro]'] + External_costs['Nox (Crops, Forests, Buildings & Ecosystems) [Euro]']

## External costs due to fine particulate matter (PM2.5) emissions

In [ ]:
External_costs['PM2.5 (Health - VOLY) [Euro]'] = Data_for_calc['PM2.5 [t]'] * Data_for_calc['Health 1: PM2.5 VOLY [Euro/t]']
External_costs['PM2.5 (Health - VSL) [Euro]'] = Data_for_calc['PM2.5 [t]'] * Data_for_calc['Health 1: PM2.5 VSL [Euro/t]']

## External costs due to coarse particulate matter (PM10) emissions

In [ ]:
External_costs['PM10 (Health - VOLY) [Euro]'] = Data_for_calc['PM10 [t]'] * Data_for_calc['Health 1: PM10 VOLY [Euro/t]']
External_costs['PM10 (Health - VSL) [Euro]'] = Data_for_calc['PM10 [t]'] * Data_for_calc['Health 1: PM10 VSL [Euro/t]']

## External costs due to Sulfur oxides (SOx) emissions

In [ ]:
External_costs['SOx (Health - VOLY) [Euro]'] = Data_for_calc['SOx [t]'] * Data_for_calc['Health 1: SOx VOLY [Euro/t]']
External_costs['SOx (Health - VSL) [Euro]'] = Data_for_calc['SOx [t]'] * Data_for_calc['Health 1: SOx VSL [Euro/t]']
External_costs['SOx (Buildings) [Euro]'] = Data_for_calc['SOx [t]'] * Data_for_calc['Buildings: Sox [Euro/t]']
External_costs['SOx (Total - VOLY) [Euro]'] = External_costs['SOx (Health - VOLY) [Euro]'] + External_costs['SOx (Buildings) [Euro]']
External_costs['SOx (Total - VSL) [Euro]'] = External_costs['SOx (Health - VSL) [Euro]'] + External_costs['SOx (Buildings) [Euro]']

## External costs due to Non-methane volatile organic compound (NMVOC) emissions

In [ ]:
External_costs['NMVOC (Health - VOLY) [Euro]'] = Data_for_calc['NMVOC [t]'] * Data_for_calc['Health 1: NMVOC VOLY [Euro/t]']
External_costs['NMVOC (Health - VSL) [Euro]'] = Data_for_calc['NMVOC [t]'] * Data_for_calc['Health 1: NMVOC VSL [Euro/t]']
External_costs['NMVOC (Forests & Crops) [Euro]'] = Data_for_calc['NMVOC [t]'] * (Data_for_calc['Crops: NMVOC [Euro/t]'] + Data_for_calc['Forests: NMVOC [Euro/t]'])
External_costs['NMVOC (Forests) [Euro]'] = Data_for_calc['NMVOC [t]'] * Data_for_calc['Forests: NMVOC [Euro/t]']
External_costs['NMVOC (Crops) [Euro]'] = Data_for_calc['NMVOC [t]'] * Data_for_calc['Crops: NMVOC [Euro/t]']
External_costs['NMVOC (Total - VOLY) [Euro]'] = External_costs['NMVOC (Health - VOLY) [Euro]'] + External_costs['NMVOC (Forests & Crops) [Euro]']
External_costs['NMVOC (Total - VSL) [Euro]'] = External_costs['NMVOC (Health - VSL) [Euro]'] + External_costs['NMVOC (Forests & Crops) [Euro]']

## External costs due to Ammonia (NH3) emissions

In [ ]:
External_costs['NH3 (Health - VOLY) [Euro]'] = Data_for_calc['NH3 [t]'] * Data_for_calc['Health 1: NH3 VOLY [Euro/t]']
External_costs['NH3 (Health - VSL) [Euro]'] = Data_for_calc['NH3 [t]'] * Data_for_calc['Health 1: NH3 VSL [Euro/t]']
External_costs['NH3 (Ecosystems) [Euro]'] = Data_for_calc['NH3 [t]'] * Data_for_calc['Ecosystems: NH3 [Euro/t]']
External_costs['NH3 (Total - VOLY) [Euro]'] = External_costs['NH3 (Health - VOLY) [Euro]'] + External_costs['NH3 (Ecosystems) [Euro]']
External_costs['NH3 (Total - VSL) [Euro]'] = External_costs['NH3 (Health - VSL) [Euro]'] + External_costs['NH3 (Ecosystems) [Euro]']

## External costs due to heavy metals

In [ ]:
External_costs['Pb (Total) [Euro]'] = Data_for_calc['Pb [t]'] * Data_for_calc['Pb cost [Euro/t]']
External_costs['Hg (Total) [Euro]'] = Data_for_calc['Hg [t]'] * Data_for_calc['Hg cost [Euro/t]']
External_costs['Cd (Total) [Euro]'] = Data_for_calc['Cd [t]'] * Data_for_calc['Cd cost [Euro/t]']
External_costs['As (Total) [Euro]'] = Data_for_calc['As [t]'] * Data_for_calc['As cost [Euro/t]']
External_costs['Ni (Total) [Euro]'] = Data_for_calc['Ni [t]'] * Data_for_calc['Ni cost [Euro/t]']
External_costs['Cr (Total) [Euro]'] = Data_for_calc['Cr [t]'] * Data_for_calc['Cr cost [Euro/t]']

## Total impact on health

In [ ]:
External_costs['Health - VOLY (Total) [Euro]'] = External_costs['Nox (Health - VOLY) [Euro]'] + External_costs['PM2.5 (Health - VOLY) [Euro]'] + External_costs['PM10 (Health - VOLY) [Euro]'] + External_costs['SOx (Health - VOLY) [Euro]'] + External_costs['NMVOC (Health - VOLY) [Euro]'] + External_costs['NH3 (Health - VOLY) [Euro]']
External_costs['Health - VSL (Total) [Euro]'] = External_costs['Nox (Health - VSL) [Euro]'] + External_costs['PM2.5 (Health - VSL) [Euro]'] + External_costs['PM10 (Health - VSL) [Euro]'] + External_costs['SOx (Health - VSL) [Euro]'] + External_costs['NMVOC (Health - VSL) [Euro]'] + External_costs['NH3 (Health - VSL) [Euro]']

## Total impact on Crops, Forests, Buildings & Ecosystems

In [ ]:
External_costs['Crops (Total) [Euro]'] = External_costs['Nox (Crops) [Euro]'] + External_costs['NMVOC (Crops) [Euro]']
External_costs['Forests (Total) [Euro]'] = External_costs['Nox (Forests) [Euro]'] + External_costs['NMVOC (Forests) [Euro]']
External_costs['Buildings (Total) [Euro]'] = External_costs['Nox (Buildings) [Euro]'] + External_costs['SOx (Buildings) [Euro]']
External_costs['Ecosystems (Total) [Euro]'] = External_costs['Nox (Ecosystems) [Euro]'] + External_costs['NH3 (Ecosystems) [Euro]']

## Air Pollutants

In [ ]:
External_costs['Air Pollutants - VOLY [Euro]'] = External_costs['Health - VOLY (Total) [Euro]'] + External_costs['Crops (Total) [Euro]'] + External_costs['Forests (Total) [Euro]'] + External_costs['Buildings (Total) [Euro]'] + External_costs['Ecosystems (Total) [Euro]']
External_costs['Air Pollutants - VSL [Euro]'] = External_costs['Health - VSL (Total) [Euro]'] + External_costs['Crops (Total) [Euro]'] + External_costs['Forests (Total) [Euro]'] + External_costs['Buildings (Total) [Euro]'] + External_costs['Ecosystems (Total) [Euro]']

## Heavy Metals

In [ ]:
External_costs['Heavy Metals (Total) [Euro]'] = External_costs['Pb (Total) [Euro]'] + External_costs['Hg (Total) [Euro]'] + External_costs['Cd (Total) [Euro]'] + External_costs['As (Total) [Euro]'] + External_costs['Ni (Total) [Euro]'] + External_costs['Cr (Total) [Euro]']

## Total External Costs

In [ ]:
External_costs_total = External_costs.loc[:,['countries','Year']]

In [ ]:
def Total(column_list, column_name, EXT_COST_TOT, EXT_COST):
  EXT_COST_TOT[column_name] = [0] * len(EXT_COST_TOT)
  for column in column_list:
    EXT_COST_TOT[column_name] += EXT_COST[column]
  return EXT_COST_TOT

In [ ]:
default_list = ['Crops (Total) [Euro]','Forests (Total) [Euro]','Buildings (Total) [Euro]','Ecosystems (Total) [Euro]','Heavy Metals (Total) [Euro]']

### Low GHG + VOLY

In [ ]:
column_list = default_list + ['Low GHG [Euro]','Health - VOLY (Total) [Euro]']
column_name = 'External cost (Low GHG + VOLY) [Euro]'
External_costs_total = Total(column_list, column_name, External_costs_total, External_costs)

### Low GHG + VSL

In [ ]:
column_list = default_list + ['Low GHG [Euro]','Health - VSL (Total) [Euro]']
column_name = 'External cost (Low GHG + VSL) [Euro]'
External_costs_total = Total(column_list, column_name, External_costs_total, External_costs)

## Central GHG + VOLY

In [ ]:
column_list = default_list + ['Central GHG [Euro]','Health - VOLY (Total) [Euro]']
column_name = 'External cost (Central GHG + VOLY) [Euro]'
External_costs_total = Total(column_list, column_name, External_costs_total, External_costs)

### Central GHG + VSL

In [ ]:
column_list = default_list + ['Central GHG [Euro]','Health - VSL (Total) [Euro]']
column_name = 'External cost (Central GHG + VSL) [Euro]'
External_costs_total = Total(column_list, column_name, External_costs_total, External_costs)

### High GHG + VOLY

In [ ]:
column_list = default_list + ['High GHG [Euro]','Health - VOLY (Total) [Euro]']
column_name = 'External cost (High GHG + VOLY) [Euro]'
External_costs_total = Total(column_list, column_name, External_costs_total, External_costs)

### High GHG + VSL

In [ ]:
column_list = default_list + ['High GHG [Euro]','Health - VSL (Total) [Euro]']
column_name = 'External cost (High GHG + VSL) [Euro]'
External_costs_total = Total(column_list, column_name, External_costs_total, External_costs)

In [ ]:
External_costs_total

,countries,Year,External cost (Low GHG + VOLY) [Euro],External cost (Low GHG + VSL) [Euro],External cost (Central GHG + VOLY) [Euro],External cost (Central GHG + VSL) [Euro],External cost (High GHG + VOLY) [Euro],External cost (High GHG + VSL) [Euro]
0,Belgium,1990,6.720881e+10,1.767501e+11,7.321097e+10,1.827522e+11,1.330896e+11,2.426309e+11
1,Belgium,1991,6.661214e+10,1.753358e+11,7.274871e+10,1.814723e+11,1.339684e+11,2.426920e+11
2,Belgium,1992,6.650597e+10,1.737718e+11,7.262903e+10,1.798949e+11,1.337139e+11,2.409798e+11
3,Belgium,1993,6.331357e+10,1.672702e+11,6.938634e+10,1.733430e+11,1.299695e+11,2.339261e+11
4,Belgium,1994,5.938863e+10,1.580216e+11,6.566302e+10,1.642959e+11,1.282575e+11,2.268905e+11
...,...,...,...,...,...,...,...,...
923,Luxembourg,2017,1.634847e+09,3.742374e+09,2.049819e+09,4.157347e+09,6.189665e+09,8.297192e+09
924,Luxembourg,2018,1.591006e+09,3.607927e+09,2.024535e+09,4.041456e+09,6.349503e+09,8.366425e+09
925,Luxembourg,2019,1.529967e+09,3.403095e+09,1.965776e+09,3.838904e+09,6.313488e+09,8.186616e+09
926,Luxembourg,2020,1.304098e+09,2.923352e+09,1.664552e+09,3.283806e+09,5.260515e+09,6.879769e+09


## Test of the importance of As, Ni and Cr

In [ ]:
Test_AsNiCr = External_costs.loc[:,['countries','Year','As (Total) [Euro]','Ni (Total) [Euro]','Cr (Total) [Euro]']]

In [ ]:
Test_AsNiCr = pd.merge(Test_AsNiCr, External_costs_total, on=['countries','Year'], how='inner')

In [ ]:
column_list_test = ['External cost (Low GHG + VOLY) [Euro]',
       'External cost (Low GHG + VSL) [Euro]',
       'External cost (Central GHG + VOLY) [Euro]',
       'External cost (Central GHG + VSL) [Euro]',
       'External cost (High GHG + VOLY) [Euro]',
       'External cost (High GHG + VSL) [Euro]']
for column in column_list_test:
  new_column = 'Proportion in % - ' + column
  Test_AsNiCr[new_column] = 100*(Test_AsNiCr['As (Total) [Euro]'] + Test_AsNiCr['Ni (Total) [Euro]'] + Test_AsNiCr['Cr (Total) [Euro]'])/Test_AsNiCr[column]
  print(f'maximum: {Test_AsNiCr[new_column].max()}% mean: {Test_AsNiCr[new_column].mean()}%')

maximum: 3.7246648688313595% mean: 0.42147167480738%
maximum: 2.5263324368158777% mean: 0.21833721853647922%
maximum: 3.100938591705925% mean: 0.35966076275602354%
maximum: 2.216707051508313% mean: 0.1985272089971547%
maximum: 2.779040487367303% mean: 0.1710723747107479%
maximum: 1.596688094303594% mean: 0.1170474331046962%


# GrDP

In [ ]:
Data_for_calc = pd.merge(Data, External_costs_total, on=['countries','Year'], how='inner')

In [ ]:
GrDP = Data_for_calc.loc[:,['countries','Year']]

In [ ]:
GrDP['GrDP (Low GHG + VOLY) [Million Euro]'] = Data_for_calc['GDP [MIO_EURO]'] - Data_for_calc['External cost (Low GHG + VOLY) [Euro]']*1e-6
GrDP['GrDP (Low GHG + VSL) [Million Euro]'] = Data_for_calc['GDP [MIO_EURO]'] - Data_for_calc['External cost (Low GHG + VSL) [Euro]']*1e-6
GrDP['GrDP (Central GHG + VOLY) [Million Euro]'] = Data_for_calc['GDP [MIO_EURO]'] - Data_for_calc['External cost (Central GHG + VOLY) [Euro]']*1e-6
GrDP['GrDP (Central GHG + VSL) [Million Euro]'] = Data_for_calc['GDP [MIO_EURO]'] - Data_for_calc['External cost (Central GHG + VSL) [Euro]']*1e-6
GrDP['GrDP (High GHG + VOLY) [Million Euro]'] = Data_for_calc['GDP [MIO_EURO]'] - Data_for_calc['External cost (High GHG + VOLY) [Euro]']*1e-6
GrDP['GrDP (High GHG + VSL) [Million Euro]'] = Data_for_calc['GDP [MIO_EURO]'] - Data_for_calc['External cost (High GHG + VSL) [Euro]']*1e-6

In [ ]:
GrDP = GrDP[GrDP.Year >= 1995]

# External Costs per Capita

In [ ]:
External_Costs_total_per_capita = Data_for_calc.loc[:,['countries','Year']]
External_Costs_total_per_capita['External cost per capita (Low GHG + VOLY) [Euro]'] = Data_for_calc['External cost (Low GHG + VOLY) [Euro]']/Data_for_calc['Population']
External_Costs_total_per_capita['External cost per capita (Low GHG + VSL) [Euro]'] = Data_for_calc['External cost (Low GHG + VSL) [Euro]']/Data_for_calc['Population']
External_Costs_total_per_capita['External cost per capita (Central GHG + VOLY) [Euro]'] = Data_for_calc['External cost (Central GHG + VOLY) [Euro]']/Data_for_calc['Population']
External_Costs_total_per_capita['External cost per capita (Central GHG + VSL) [Euro]'] = Data_for_calc['External cost (Central GHG + VSL) [Euro]']/Data_for_calc['Population']
External_Costs_total_per_capita['External cost per capita (High GHG + VOLY) [Euro]'] = Data_for_calc['External cost (High GHG + VOLY) [Euro]']/Data_for_calc['Population']
External_Costs_total_per_capita['External cost per capita (High GHG + VSL) [Euro]'] = Data_for_calc['External cost (High GHG + VSL) [Euro]']/Data_for_calc['Population']

In [ ]:
External_Costs_total_per_capita = External_Costs_total_per_capita[External_Costs_total_per_capita.Year >= 1995]

In [ ]:
External_Costs_total_per_capita

,countries,Year,External cost per capita (Low GHG + VOLY) [Euro],External cost per capita (Low GHG + VSL) [Euro],External cost per capita (Central GHG + VOLY) [Euro],External cost per capita (Central GHG + VSL) [Euro],External cost per capita (High GHG + VOLY) [Euro],External cost per capita (High GHG + VSL) [Euro]
130,Belgium,1995,5809.752785,15126.270022,6437.005486,15753.522723,12694.597904,22011.115141
131,Bulgaria,1995,6547.418580,15632.531546,6825.054064,15910.167030,9594.798541,18679.911507
132,Cyprus,1995,3198.346803,4249.757902,3647.398041,4698.809140,8127.218725,9178.629824
133,Czechia,1995,7074.750014,18650.643742,7685.400380,19261.294107,13777.364739,25353.258466
134,Germany,1995,3113.610560,7896.955262,3678.947411,8462.292113,9318.855515,14102.200218
...,...,...,...,...,...,...,...,...
923,Switzerland,2020,1027.102517,2394.388378,1234.416962,2601.702824,3302.625360,4669.911221
924,Luxembourg,2020,2082.864142,4669.085661,2658.570729,5244.792248,8401.929303,10988.150822
925,Austria,2021,1350.152538,3197.763538,1665.790438,3513.401437,4814.654242,6662.265242
926,Switzerland,2021,1022.215672,2362.916236,1232.322243,2573.022807,3328.385413,4669.085977


# GrDP per capita

In [ ]:
GrDP_per_capita = Data_for_calc.loc[:,['countries','Year']]

In [ ]:
GrDP_per_capita['GrDP per capita (Low GHG + VOLY) [Euro]'] = 1e6*GrDP['GrDP (Low GHG + VOLY) [Million Euro]']/Data_for_calc['Population']
GrDP_per_capita['GrDP per capita (Low GHG + VSL) [Euro]'] = 1e6*GrDP['GrDP (Low GHG + VSL) [Million Euro]']/Data_for_calc['Population']
GrDP_per_capita['GrDP per capita (Central GHG + VOLY) [Euro]'] = 1e6*GrDP['GrDP (Central GHG + VOLY) [Million Euro]']/Data_for_calc['Population']
GrDP_per_capita['GrDP per capita (Central GHG + VSL) [Euro]'] = 1e6*GrDP['GrDP (Central GHG + VSL) [Million Euro]']/Data_for_calc['Population']
GrDP_per_capita['GrDP per capita (High GHG + VOLY) [Euro]'] = 1e6*GrDP['GrDP (High GHG + VOLY) [Million Euro]']/Data_for_calc['Population']
GrDP_per_capita['GrDP per capita (High GHG + VSL) [Euro]'] = 1e6*GrDP['GrDP (High GHG + VSL) [Million Euro]']/Data_for_calc['Population']

In [ ]:
GrDP_per_capita = GrDP_per_capita[GrDP_per_capita.Year >= 1995]

In [ ]:
GrDP_per_capita

,countries,Year,GrDP per capita (Low GHG + VOLY) [Euro],GrDP per capita (Low GHG + VSL) [Euro],GrDP per capita (Central GHG + VOLY) [Euro],GrDP per capita (Central GHG + VSL) [Euro],GrDP per capita (High GHG + VOLY) [Euro],GrDP per capita (High GHG + VSL) [Euro]
130,Belgium,1995,15931.512813,6614.995576,15304.260113,5987.742876,9046.667695,-269.849542
131,Bulgaria,1995,-4825.325288,-13910.438254,-5102.960773,-14188.073739,-7872.705250,-16957.818216
132,Cyprus,1995,8571.116739,7519.705639,8122.065501,7070.654401,3642.244816,2590.833717
133,Czechia,1995,-2622.249952,-14198.143679,-3232.900317,-14808.794044,-9324.864676,-20900.758404
134,Germany,1995,21139.982551,16356.637849,20574.645700,15791.300998,14934.737596,10151.392893
...,...,...,...,...,...,...,...,...
923,Switzerland,2020,74587.597079,73220.311217,74380.282633,73012.996772,72312.074235,70944.788374
924,Luxembourg,2020,100973.317859,98387.096340,100397.611272,97811.389752,94654.252698,92068.031178
925,Austria,2021,44016.089828,42168.478829,43700.451929,41852.840929,40551.588124,38703.977125
926,Switzerland,2021,78279.354054,76938.653490,78069.247484,76728.546920,75973.184314,74632.483750


# Figures

## Interactive Map

In [ ]:
GrDP_per_capita_2021 = GrDP_per_capita.loc[GrDP_per_capita["Year"]==2021, ["countries", 'GrDP per capita (High GHG + VSL) [Euro]']]
External_Costs_total_per_capita_2021 = External_Costs_total_per_capita.loc[External_Costs_total_per_capita["Year"]==2021, ["countries", "External cost per capita (High GHG + VSL) [Euro]"]]

In [ ]:
map_grdp = px.choropleth(
    GrDP_per_capita_2021,                                         # dataframe as argument
    locations="countries",                                  # column name with location
    locationmode="country names",                         # location method
    color="GrDP per capita (High GHG + VSL) [Euro]",                       # color countries with altruism value
    color_continuous_scale=px.colors.sequential.Turbo     # choice of colors
)

map_grdp.update_layout(
    title={                                # Add a title, centered on top
        'text': "Green Domestic Product per capita",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})


map_grdp.update_geos(showocean=True,              # fill ocean with colors
                     oceancolor="LightBlue",      # specify color of oceans
                     projection_type="orthographic")  # specify projection type

map_grdp.show()

In [ ]:
map_gps = px.choropleth(
    GrDP_per_capita_2021,                                 # dataframe as argument
    locations="countries",                       # column name with location
    locationmode="country names",                   # location method
    color="GrDP per capita (High GHG + VSL) [Euro]",                       # color countries with altruism value
    color_continuous_scale=px.colors.sequential.Greens,
    scope='europe',# choice of colors
     height=800,  # Define the height of the figure
     width=1000
)
map_gps.update_layout(
    title={                                # Add a title, centered on top
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
    geo=dict(
        center=dict(lat=50, lon=10),  # Set center of the map
        projection_scale=1.5  # Adjust the zoom level
    ))
map_gps.show()

In [ ]:
map_gps = px.choropleth(
    External_Costs_total_per_capita_2021,                                 # dataframe as argument
    locations="countries",                       # column name with location
    locationmode="country names",                   # location method
    color="External cost per capita (High GHG + VSL) [Euro]",                       # color countries with altruism value
    color_continuous_scale=px.colors.sequential.Reds,
    scope='europe',# choice of colors
     height=800,  # Define the height of the figure
     width=1000
)
map_gps.update_layout(
    title={                                # Add a title, centered on top
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
    geo=dict(
        center=dict(lat=50, lon=10),  # Set center of the map
        projection_scale=1.5  # Adjust the zoom level
    ))
map_gps.show()

## Bar Chart

In [ ]:
# Import bokeh plotting package (must have explicit import of submodules)
import bokeh.io
import bokeh.models
import bokeh.plotting

# Enable viewing Bokeh plots in the notebook
bokeh.io.output_notebook()

In [ ]:
GrDP_per_capita.columns

Index(['countries', 'Year', 'GrDP per capita (Low GHG + VOLY) [Euro]',
       'GrDP per capita (Low GHG + VSL) [Euro]',
       'GrDP per capita (Central GHG + VOLY) [Euro]',
       'GrDP per capita (Central GHG + VSL) [Euro]',
       'GrDP per capita (High GHG + VOLY) [Euro]',
       'GrDP per capita (High GHG + VSL) [Euro]'],
      dtype='object')

In [ ]:
grdp_country_2021_BarChart = (GrDP_per_capita.loc[GrDP_per_capita["Year"]==2021, ["countries","GrDP per capita (High GHG + VSL) [Euro]"]]
                     .sort_values(by=["GrDP per capita (High GHG + VSL) [Euro]"], ascending=False))

bar_grdp = bokeh.plotting.figure(
    frame_height=400,
    frame_width=400,
    x_axis_label='GrDP per capita in 2021 (in Euros)',
    y_range = grdp_country_2021_BarChart["countries"][::-1],
)

source_data = bokeh.models.ColumnDataSource(grdp_country_2021_BarChart)

bar_grdp.hbar(
    source=source_data,        # our data
    y="countries",               # Country on the y-axis
    right='GrDP per capita (High GHG + VSL) [Euro]', #GrDP per capita on the x-axis, to the right
    height=0.6                 # height of bars
)

# Turn off gridlines on categorical axis
bar_grdp.ygrid.grid_line_color = None

# Start axes at origin on quantitative axis
bar_grdp.x_range.start = 0

bokeh.io.show(bar_grdp)

In [ ]:
External_costs_per_capita_2021_BarChart = (External_Costs_total_per_capita.loc[External_Costs_total_per_capita["Year"]==2021, ["countries","External cost per capita (High GHG + VSL) [Euro]"]]
                     .sort_values(by=["External cost per capita (High GHG + VSL) [Euro]"], ascending=False))

bar_grdp = bokeh.plotting.figure(
    frame_height=400,
    frame_width=400,
    x_axis_label='External Costs per capita in 2021 (in Euros)',
    y_range = External_costs_per_capita_2021_BarChart["countries"][::-1],
)

source_data = bokeh.models.ColumnDataSource(External_costs_per_capita_2021_BarChart)

bar_grdp.hbar(
    source=source_data,        # our data
    y="countries",               # Country on the y-axis
    right='External cost per capita (High GHG + VSL) [Euro]', #GrDP per capita on the x-axis, to the right
    height=0.6                 # height of bars
)

# Turn off gridlines on categorical axis
bar_grdp.ygrid.grid_line_color = None

# Start axes at origin on quantitative axis
bar_grdp.x_range.start = 0

bokeh.io.show(bar_grdp)

## Line Chart

In [ ]:
LineChart = GrDP.loc[GrDP.countries.isin(['Switzerland'])]

evol_grdp_ch = px.line(LineChart,     # data frame as argument
                       x="Year",    # x-axis: years
                       y="GrDP (High GHG + VSL) [Million Euro]",    # y-axis: GrDP
                       title="Evolution of Green Domestic Product in Switzerland",  # title!
                       range_y= [0,700000]      # specify range to avoid truncation!
                      )
evol_grdp_ch.show()

## Pie chart

In [ ]:
PieChart_Pollutant = (External_costs.loc[(External_costs["countries"]=="France") & (External_costs["Year"]==2021),
                            ['Central GHG [Euro]', 'Air Pollutants - VOLY [Euro]','Heavy Metals (Total) [Euro]']])

PieChart_Pollutant = PieChart_Pollutant.transpose().rename(columns={PieChart_Pollutant.index[0]: "External Cost"})
PieChart_Pollutant["Pollutant"]=["GHG","Air Pollutant","Heavy Metal"]

pie_cost = px.pie(PieChart_Pollutant,               # data frame as input
             values="External Cost",     # values to create the pie chart (data frame column name)
             names="Pollutant",          # labels (data frame column name)
             title="Share of external cost per pollutant in France in 2021")  # title!
pie_cost.show()

In [ ]:
PieChart_AirPol_Impact = (External_costs.loc[(External_costs["countries"]=="France") & (External_costs["Year"]==2021),
                            ['Health - VSL (Total) [Euro]', 'Crops (Total) [Euro]','Forests (Total) [Euro]','Buildings (Total) [Euro]','Ecosystems (Total) [Euro]']])

PieChart_AirPol_Impact = PieChart_AirPol_Impact.transpose().rename(columns={PieChart_AirPol_Impact.index[0]: "External Cost"})
PieChart_AirPol_Impact["Impact"]=["Health","Crops","Forests","Buildings","Ecosystems"]

pie_cost = px.pie(PieChart_AirPol_Impact,               # data frame as input
             values="External Cost",     # values to create the pie chart (data frame column name)
             names="Impact",          # labels (data frame column name)
             title="Share of external cost per impact of air pollutant in France in 2021")  # title!
pie_cost.show()

In [ ]:
PieChart_AirPol = (External_costs.loc[(External_costs["countries"]=="France") & (External_costs["Year"]==2021),
                            ['Nox (Total - VOLY) [Euro]', 'PM2.5 (Health - VOLY) [Euro]', 'PM10 (Health - VOLY) [Euro]', 'SOx (Total - VOLY) [Euro]','NMVOC (Total - VOLY) [Euro]','NH3 (Total - VOLY) [Euro]']])

PieChart_AirPol = PieChart_AirPol.transpose().rename(columns={PieChart_AirPol.index[0]: "External Cost"})
PieChart_AirPol["Air Pollutant"]=["NOx","PM2.5","PM10","SOx","NMVOC","NH3"]

pie_cost = px.pie(PieChart_AirPol,               # data frame as input
             values="External Cost",     # values to create the pie chart (data frame column name)
             names="Air Pollutant",          # labels (data frame column name)
             title="Share of external cost per air pollutant in France in 2021")  # title!
pie_cost.show()

In [ ]:
PieChart_HeavyMetals = (External_costs.loc[(External_costs["countries"]=="France") & (External_costs["Year"]==2021),
                            ['Pb (Total) [Euro]', 'Hg (Total) [Euro]','Cd (Total) [Euro]','As (Total) [Euro]','Ni (Total) [Euro]','Cr (Total) [Euro]']])

PieChart_HeavyMetals = PieChart_HeavyMetals.transpose().rename(columns={PieChart_HeavyMetals.index[0]: "External Cost"})
PieChart_HeavyMetals["Heavy Metal"]=["Pb","Hg","Cd","As","Ni","Cr"]

pie_cost = px.pie(PieChart_HeavyMetals,               # data frame as input
             values="External Cost",     # values to create the pie chart (data frame column name)
             names="Heavy Metal",          # labels (data frame column name)
             title="Share of external cost per heavy metal in France in 2021")  # title!
pie_cost.show()

## Area Chart

In [ ]:
AreaChart = (External_costs.loc[(External_costs["countries"]=="Switzerland"),
                            ["Year", 'Central GHG [Euro]', 'Air Pollutants - VOLY [Euro]','Heavy Metals (Total) [Euro]']])

In [ ]:
obs = len(AreaChart)
pol = ["GHG"]*obs+["Air Pollutant"]*obs+["Heavy Metal"]*obs
year = list(AreaChart["Year"])*3
cost = list(AreaChart['Central GHG [Euro]'])+list(AreaChart['Air Pollutants - VOLY [Euro]'])+list(AreaChart['Heavy Metals (Total) [Euro]'])

In [ ]:
dico_cost_ch ={"Year":year, "Pollutant":pol, "External cost":cost}
df_cost_ch = pd.DataFrame(data=dico_cost_ch)

In [ ]:
area_cost = px.area(df_cost_ch,         # data frame as input
                   x="Year",            # Years on the x-axis (data frame column name)
                   y="External cost",   # External cost value on the y-axis (data frame column name)
                   color="Pollutant",   # Color by group of pollutants    (data frame column name)
                   title = "Evolution of external costs per group of pollutants in Switzerland, in Euros"  #Title
                  )
area_cost.show()

# Interactive figures

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact
import plotly.express as px

## Line Chart

In [ ]:
# Function to plot data
def plot_country_data(country, column):
    plt.figure(figsize=(10, 6))
    plt.plot(GrDP[GrDP['countries'] == country]['Year'], GrDP[GrDP['countries'] == country][column], linestyle='-')
    plt.title(f'{column} for {country}')
    plt.xlabel('Year')
    plt.ylabel(f'{column}')
    plt.grid(True)
    plt.show()

# Get unique countries and columns
unique_countries = GrDP['countries'].unique()
unique_columns = GrDP.columns[2:]  # Exclude 'countries' and 'Year'

# Create interactive plot using widgets
interact(plot_country_data, country=widgets.Dropdown(options=unique_countries, description='Country:'), column=widgets.Dropdown(options=unique_columns, description='Column:'))

interactive(children=(Dropdown(description='Country:', options=('Belgium', 'Bulgaria', 'Cyprus', 'Czechia', 'G…

<function __main__.plot_country_data(country, column)>

In [ ]:
# Function to plot data
def plot_country_data(country, column):
    plt.figure(figsize=(10, 6))
    plt.plot(GrDP_per_capita[GrDP_per_capita['countries'] == country]['Year'], GrDP_per_capita[GrDP_per_capita['countries'] == country][column], linestyle='-')
    plt.title(f'{column} for {country}')
    plt.xlabel('Year')
    plt.ylabel(f'{column}')
    plt.grid(True)
    plt.show()

# Get unique countries and columns
unique_countries = GrDP_per_capita['countries'].unique()
unique_columns = GrDP_per_capita.columns[2:]  # Exclude 'countries' and 'Year'

# Create interactive plot using widgets
interact(plot_country_data, country=widgets.Dropdown(options=unique_countries, description='Country:'), column=widgets.Dropdown(options=unique_columns, description='Column:'))

interactive(children=(Dropdown(description='Country:', options=('Belgium', 'Bulgaria', 'Cyprus', 'Czechia', 'G…

<function __main__.plot_country_data(country, column)>

In [ ]:
# Function to plot data
def plot_country_data(country, column):
    plt.figure(figsize=(10, 6))
    plt.plot(External_costs_total[External_costs_total['countries'] == country]['Year'], External_costs_total[External_costs_total['countries'] == country][column], linestyle='-')
    plt.title(f'{column} for {country}')
    plt.xlabel('Year')
    plt.ylabel(f'{column}')
    plt.grid(True)
    plt.show()

# Get unique countries and columns
unique_countries = External_costs_total['countries'].unique()
unique_columns = External_costs_total.columns[2:]  # Exclude 'countries' and 'Year'

# Create interactive plot using widgets
interact(plot_country_data, country=widgets.Dropdown(options=unique_countries, description='Country:'), column=widgets.Dropdown(options=unique_columns, description='Column:'))

interactive(children=(Dropdown(description='Country:', options=('Belgium', 'Bulgaria', 'Cyprus', 'Czechia', 'G…

<function __main__.plot_country_data(country, column)>

In [ ]:
# Function to plot data
def plot_country_data(country, column):
    plt.figure(figsize=(10, 6))
    plt.plot(External_Costs_total_per_capita[External_Costs_total_per_capita['countries'] == country]['Year'], External_Costs_total_per_capita[External_Costs_total_per_capita['countries'] == country][column], linestyle='-')
    plt.title(f'{column} for {country}')
    plt.xlabel('Year')
    plt.ylabel(f'{column}')
    plt.grid(True)
    plt.show()

# Get unique countries and columns
unique_countries = External_Costs_total_per_capita['countries'].unique()
unique_columns = External_Costs_total_per_capita.columns[2:]  # Exclude 'countries' and 'Year'

# Create interactive plot using widgets
interact(plot_country_data, country=widgets.Dropdown(options=unique_countries, description='Country:'), column=widgets.Dropdown(options=unique_columns, description='Column:'))

interactive(children=(Dropdown(description='Country:', options=('Belgium', 'Bulgaria', 'Cyprus', 'Czechia', 'G…

<function __main__.plot_country_data(country, column)>

## Pie Chart

In [ ]:
# Function to plot data
def plot_PieChart(country, year):
    PieChart_Pollutant = (External_costs.loc[(External_costs["countries"]==country) & (External_costs["Year"]==year),
                            ['Central GHG [Euro]', 'Air Pollutants - VOLY [Euro]','Heavy Metals (Total) [Euro]']])
    PieChart_Pollutant = PieChart_Pollutant.transpose().rename(columns={PieChart_Pollutant.index[0]: "External Cost"})
    PieChart_Pollutant["Pollutant"]=["GHG","Air Pollutant","Heavy Metal"]
    pie_cost = px.pie(PieChart_Pollutant,               # data frame as input
             values="External Cost",     # values to create the pie chart (data frame column name)
             names="Pollutant",          # labels (data frame column name)
             title="Share of external cost per pollutant in France in 2021")  # title!
    pie_cost.show()

# Get unique countries and columns
unique_countries = External_Costs_total_per_capita['countries'].unique()
unique_years = External_Costs_total_per_capita['Year'].unique()

# Create interactive plot using widgets
interact(plot_PieChart, country=widgets.Dropdown(options=unique_countries, description='Country:'), year=widgets.Dropdown(options=unique_years, description='Year:'))

interactive(children=(Dropdown(description='Country:', options=('Belgium', 'Bulgaria', 'Cyprus', 'Czechia', 'G…

<function __main__.plot_PieChart(country, year)>

## Map

In [ ]:
def plot_map(year, column):
    fig = px.choropleth(GrDP_per_capita,
                        locations='countries',
                        locationmode='country names',
                        color=column,
                        hover_name='countries',
                        animation_frame='Year',
                        title=f"{column} by Country in {year}",
                        color_continuous_scale=px.colors.sequential.Greens,
                        scope='europe',
                        height=800,
                       width=1000 )
    fig.show()


# Example usage
plot_map(2021, 'GrDP per capita (Low GHG + VOLY) [Euro]')

In [ ]:
def plot_map(year, column):
    fig = px.choropleth(External_Costs_total_per_capita,
                        locations='countries',
                        locationmode='country names',
                        color=column,
                        hover_name='countries',
                        animation_frame='Year',
                        title=f"{column} by Country in {year}",
                        color_continuous_scale=px.colors.sequential.Reds,
                        scope='europe',
                        height=800,
                       width=1000 )
    fig.show()


# Example usage
plot_map(2021, 'External cost per capita (High GHG + VSL) [Euro]')

In [ ]:
def plot_map(column):
    fig = px.choropleth(External_Costs_total_per_capita,
                        locations='countries',
                        locationmode='country names',
                        color=column,
                        hover_name='countries',
                        animation_frame='Year',
                        title=f"{column} by Country",
                        color_continuous_scale=px.colors.sequential.Reds,
                        scope='europe',
                        height=800,
                       width=1000 )
    fig.show()

# Get unique countries and columns
unique_columns = External_Costs_total_per_capita.columns[2:]  # Exclude 'countries' and 'Year'

# Create interactive plot using widgets
interact(plot_map, column=widgets.Dropdown(options=unique_columns, description='Column:'))

interactive(children=(Dropdown(description='Column:', options=('External cost per capita (Low GHG + VOLY) [Eur…

<function __main__.plot_map(column)>